In [27]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from tqdm import tqdm
import pandas as pd

# 모델 초기화
model = BertModel.from_pretrained('skt/kobert-base-v1')  # 한국어 BERT 모델을 로드합니다.
kw_model = KeyBERT(model)  # KeyBERT 모델을 초기화합니다. 이 모델은 키워드 추출에 사용됩니다.
kiwi = Kiwi()  # Kiwi 한국어 형태소 분석기를 초기화합니다.

# Load the CSV file again
data = pd.read_csv('preprocessed_total_reviews_beauty.csv')

# Extracting each row from the 'preprocessed_content' column and storing it in a list
content_list = data['preprocessed_content'].tolist()

In [40]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)  # 텍스트를 분석하여 형태소 분석 결과를 얻습니다.
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):  # 명사와 외래어를 추출합니다.
            results.append(token)
    return results

# 키워드 추출
extracted_keywords = []
for text in tqdm(data['content']):  # 데이터의 'content' 열을 순회하며 키워드를 추출합니다.
    # 명사 추출
    nouns = noun_extractor(text)
    after_kiwi_text = ' '.join(nouns)
    
    # KeyBERT를 사용하여 키워드 추출
    keywords = kw_model.extract_keywords(after_kiwi_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    extracted_keywords.append(keywords)

# 추출된 키워드에서 단어만 추출
words_only = [[keyword[0] for keyword in keywords_list] for keywords_list in extracted_keywords]

# 'preprocessed_category'와 'preprocessed_productname' 열을 합쳐서 새로운 열 생성
data['Extracted_Keywords'] = data['preprocessed_category'] + " " + data['preprocessed_productname']

# 기존 'hashtags' 열 유지
# 'hashtags' 열은 이전과 같이 추출된 키워드를 해시태그 형태로 저장합니다.
data['hashtags'] = [' '.join(keyword_list) for keyword_list in words_only]

# 나머지 부분은 이전과 동일합니다...

# 새로운 DataFrame 열 생성 및 수정
data.rename(columns={'likes': 'importance'}, inplace=True)

# datetime으로 정렬
data['datetime'] = pd.to_datetime(data['datetime'])
data.sort_values(by='datetime', inplace=True)

100%|██████████| 1000/1000 [00:13<00:00, 73.91it/s]


In [41]:
#각 단어 마다 , 추가
def add_commas(string):
    # Split the string by spaces and join with commas
    return ', '.join(string.split())

# commas 함수를 hashtags와 
data['hashtags'] = data['hashtags'].apply(add_commas)
data['Extracted_Keywords'] = data['Extracted_Keywords'].apply(add_commas)

# 처음과 끝에 ' [ '와 ' ] '추가 하는 함수를 hashtags와 Extracted_Keyword에 적용
def add_brackets(string):
    return f'[{string}]'

# 괄호 추가 함수를 hashtags와 Extracted_Keyword에 적용
data['hashtags'] = data['hashtags'].apply(add_brackets)
data['Extracted_Keywords'] = data['Extracted_Keywords'].apply(add_brackets)

# Display the modified dataframe
data.head()

,category,productname,content,importance,datetime,preprocessed_category,preprocessed_productname,preprocessed_content,Extracted_Keywords,hashtags
701,스킨케어,[동국제약] 센텔리안24 마데카 마이크로 스타톡 세럼 50ml,요즘 컬리에서 행사가로 올려주시는 화장품들 덕분에\r\n심적 호강을 누리고 있는 저...,120,2021-10-23,스킨케어,동국제약 마데카 마이크로 스타톡 세럼,요즘 컬리 에서 행사 가로 올려주시는 화장품 들 덕분 에 심 적 호강 누리 고 있는...,"[스킨케어, 동국제약, 마데카, 마이크로, 스타톡, 세럼]","[화장품, 쫀득이, 향도]"
950,클렌징,[아리얼] 스트레스 릴리빙 퓨어풀 클렌징 티슈 80매,저는 조금 클렌징만 덜 되도 바로 피부가 표시가 나는 ^^ 그런 민감하고 예민한 트...,16,2022-03-02,클렌징,아리얼 스트레스 릴리빙 퓨어풀 클렌징 티슈,저 는 조금 클렌징 만 덜 되도 바로 피부 가 표시 가 나 는 그런 민감하고 예민한...,"[클렌징, 아리얼, 스트레스, 릴리빙, 퓨어풀, 클렌징, 티슈]","[피부, 얼굴, 표시]"
411,메이크업,[리르] 포크 아이브로우 틴트 4종,아침마다 눈썹 그리기 너무 귀찮다 생각하는 와중\r\n마침 세일도 하고 괜찮을것같아...,12,2022-04-21,메이크업,리르 포크 아이브로우 틴트,아침 마다 눈썹 그리기 너무 귀찮다 생각 하는 와중 마침 세 일도 하고 괜찮을것 같...,"[메이크업, 리르, 포크, 아이브로우, 틴트]","[뭉퉁, 짱구, 조금]"
740,스킨케어,[비플레인] 멀티 비타민 앰플,앰플을 찾고있던 와중에 비플레인 멀티비타민 앰플이 세일하길래 바로 구매했어요!\r\...,6,2022-05-25,스킨케어,비플레인 멀티 비타민 앰플,앰플 찾고있던 와중 에 비 플레인 멀티 비타민 앰플 세 일하길래 바로 구매 했어요 ...,"[스킨케어, 비플레인, 멀티, 비타민, 앰플]","[비타민, 영양, 피부]"
410,메이크업,[리르] 포크 아이브로우 틴트 4종,"어제 오후에 구입하고 오늘 새벽에 받았어요. 뷰티제품도 새벽 배송인지 몰랐는데, 아...",23,2022-06-15,메이크업,리르 포크 아이브로우 틴트,어제 오후 에 구입 하고 오늘 새벽 에 받았어요 뷰티 제품 도 새벽 배송 인지 몰랐...,"[메이크업, 리르, 포크, 아이브로우, 틴트]","[컬러, 그레이, 헤어]"


In [47]:
# 특정 열 선택 및 새 CSV 파일로 저장
final_data = data[['datetime', 'hashtags', 'importance', 'Extracted_Keywords']]

# importance가 0 보다 큰 값만 저장
final_data = final_data[final_data['importance'] > 0]

final_csv_path = 'C:/Users/kgw57/PycharmProjects/data_contest/keyword_review/datetime_hashtags_importance_keywords_beauty.csv'
final_data.to_csv(final_csv_path, index=False)  # 선택된 열을 새 CSV 파일로 저장합니다.

final_csv_path

'C:/Users/kgw57/PycharmProjects/data_contest/keyword_review/datetime_hashtags_importance_keywords_beauty.csv'